In [ ]:
!pip install -U transformers==3.0.0

In [ ]:
!python -m nltk.downloader punkt

In [ ]:
!git clone https://github.com/patil-suraj/question_generation.git

In the next block, enter the google drive id of the input and reference files to be loaded as the context and rename the file variables to be their path. 
Alternatively, in case a local file being used, comment out the gdown line and set file to be the path of the context file.

Also, set the output_file_name to be the path to the output file.

In [ ]:
import os
os.chdir("/content/")
!gdown '1KJ66LbUiPH-uLzuSMvJRTGjXrM-XnHCY'
!gdown '1KHj25NWCVCZu7ViHz8bUAKL990ge_Jd4'
file = "/content/context.txt"
ref_file = "/content/references.txt"
output_file_name = "/content/t5_small_hypothesis.txt"

In [ ]:
os.chdir("question_generation")
from pipelines import pipeline
nlp = pipeline("multitask-qa-qg")

In [ ]:
f = open(file,'r')
list = []
for line in f:
  s = ""
  for c in line:
    if (c != '"'):
      s += c
  print(s)
  list += nlp(s)
f.close()

In [ ]:
questions = [x['question']+'\n' for x in list]
answers = [x['answer'] for x in list]
print(len(list))
print(list)
print(questions)


In [ ]:
out = open(output_file_name, "w")
out.writelines(questions)
out.close()

In [ ]:

f3 = open(ref_file, "r")
references = []
blref = []
for line in f3:
#     print(line)
    blref.insert(0, line)
    references.insert(0, line.split())
print(len(references))
f3.close()

Now, we evaluate the generated questions using the BLEU and METEOR scores of the generated questions with respect to the reference questions. For this purpose we use the nltk library, the use of which is illustrated here.

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)
METEOR = nltk.translate.meteor_score.meteor_score([reference], hypothesis)
print(f"{line}METEOR : {METEOR}\n")

In [ ]:
f2 = open(output_file_name, "r")
i = 0
BLEUdict = {}
METEORdict = {}
for line in f2:
#     print(line)
    i = i + 1
    hypothesis = line
    BLEUscore = nltk.translate.bleu_score.sentence_bleu(blref, hypothesis)
    BLEUdict[line] = BLEUscore
    print(f"{i} : {line}BLEU4 : {BLEUscore}\n")
    METEORscore = nltk.translate.meteor_score.meteor_score(references, hypothesis.split())
    METEORdict[line] = METEORscore
    print(f"METEOR : {METEORscore}\n")
f2.close()

In [ ]:
sorted(BLEUdict.items(), key=lambda item: item[-1], reverse = True)

In [ ]:
sorted(METEORdict.items(), key=lambda item: item[1], reverse = True)

In [ ]:
print(sum(BLEUdict.values()) / len(BLEUdict.values()))
print(sum(METEORdict.values()) / len(METEORdict.values()))
print(sum(sorted(BLEUdict.values(),reverse = True)[0:12]) / 12)
print(sum(sorted(METEORdict.values(),reverse = True)[0:12]) / 12)